In [ ]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

In [ ]:
def load_stock_base_msg():
    stock_data_path = 'E:/Download/QQ/28814405/FileRecv/'
    stock_data_cvs = stock_data_path + 'trader_report_1.csv'
    #
    stat = pd.read_csv(stock_data_cvs,encoding='gbk',index_col=['code'])
    return stat

In [ ]:
def get_date_period_staticstics(begtime,endtime):
    #
    stock_data_path = 'E:/project/pychram/traderesp/strategy/output-csv/'
    stock_data_cvs = stock_data_path + 'output-stock-history-adjust-m26-' \
                                     + begtime.strftime('%Y%m%d') + '-' + endtime.strftime('%Y%m%d') + '.csv'
    #
    stock_data_path = 'E:/Download/QQ/28814405/FileRecv/'
    stock_data_cvs = stock_data_path + 'trader_report_1.csv'
    stat = pd.read_csv(stock_data_cvs,parse_dates=3,encoding='gbk')
    
    #逐笔利润
    stat['profit'] = stat['sell_price'] - stat['buy_price']
    #
    action_num = len(stat)
    suc_num = stat[stat['profit'] > 0.00 ].count() 
    fail_num = action_num - suc_num
    #
    tmp_profit = stat.groupby(['code']).sum()
    year_action_num = len(tmp_profit)
    tmp_profit['profit_rate'] = (tmp_profit['sell_price'] - tmp_profit['buy_price']) / tmp_profit['buy_price']
    year_suc_num = tmp_profit[tmp_profit['profit_rate'] > 0.00].count()
    year_fail_num = year_action_num - year_suc_num
    #
    year_suc_num_gt_percet_5 = tmp_profit[tmp_profit['profit_rate'] > 0.05].count()
    #
    stat_sum_all = stat.sum()
    code_unique_num = len(stat['code'].unique())
    s_ge_b_num = stat[stat['sell_price'] > stat['buy_price']].count()
    #
    print '-----  ',begtime,'  ----  ',endtime,'  -----'
    print '股票总数(排重) :',year_action_num
    print '买卖总次数(一次买卖算一次) :',action_num
    print '其中获利次数 :',suc_num['profit']
    print '其中亏损次数 :',fail_num['profit']
    print '获利次数比率 :',suc_num['profit'] / action_num
    print '亏损次数比率 :',fail_num['profit'] / action_num
    print '年度获利股票数 :',year_suc_num['profit_rate']
    print '年度亏损股票数 :',year_fail_num['profit_rate']
    print '年度获利股票数比率 :',year_suc_num['profit_rate'] / year_action_num
    print '年度亏损股票数比率 :',year_fail_num['profit_rate'] / year_action_num
    print '年度获利股票数( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate']
    print '年度获利股票数比率( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate'] / year_action_num
    print '---------------------------------------------------------------------'
    #
    return tmp_profit[tmp_profit['profit_rate'] > 0.00].sort_values('profit_rate',ascending=False),tmp_profit,stat

In [ ]:
def show_date_staticstics():
    code1 = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))
    code2 = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))
    code3 = set(code1.index) & set(code2.index)
    year_2_num = len(code3)
    print '连续2年上涨股票数(获利股票数) :',year_2_num
    print '连续2年上涨股票数比率 :',year_2_num / len(code1)
    #--------------------------------------------------------
    code1_gt_percent_5 =  code1[code1['profit_rate'] > 0.05]
    code2_gt_percent_5 =  code2[code2['profit_rate'] > 0.05]
    code3_gt_percent_5 =  set(code1_gt_percent_5.index) & set(code2_gt_percent_5.index)
    print '连续2年上涨股票数(获利股票数 大于 5%) :',len(code3_gt_percent_5)
    print '连续2年上涨股票数比率(获利大于 5%) :',len(code3_gt_percent_5) / len(code1)
    #--------------------------------------------------------
    code4 = code1.index[0:100]
    code5 = code2.index[0:100]
    code6 = set(code4) & set(code5)
    year_2_num_from_100 = len(code6)
    print '连续2年上涨股票数(Top 100) :',year_2_num_from_100
    code7 = code1.index[0:200]
    code8 = code2.index[0:200]
    code9 = set(code7) & set(code8)
    year_2_num_from_200 = len(code9)
    print '连续2年上涨股票数(Top 200) :',year_2_num_from_200
    code10 = code1.index[0:300]
    code11 = code2.index[0:300]
    code12 = set(code10) & set(code11)
    year_2_num_from_300 = len(code12)
    print '连续2年上涨股票数(Top 300) :',year_2_num_from_300
    code13 = code1.index[0:400]
    code14 = code2.index[0:400]
    code15 = set(code13) & set(code14)
    year_2_num_from_400 = len(code15)
    print '连续2年上涨股票数(Top 400) :',year_2_num_from_400
    code16 = code1.index[0:500]
    code17 = code2.index[0:500]
    code18 = set(code16) & set(code17)
    year_2_num_from_500 = len(code18)
    print '连续2年上涨股票数(Top 500) :',year_2_num_from_500
    #
    code_bing = pd.merge(code1,code2,left_index=True, right_index=True)
    return (code_bing,code_bing[(code_bing['profit_rate_x'] > 0.05) & (code_bing['profit_rate_y'] > 0.05)])

In [ ]:
code_base_msg = load_stock_base_msg()
code_profit_rate_ge_0, code_profit_rate_ge_5 = show_date_staticstics()
code_profit_rate_ge_5 = code_profit_rate_ge_5[['profit_rate_x','profit_rate_y']]

In [ ]:
code_bing = pd.merge(code_base_msg,code_profit_rate_ge_5,left_index=True, right_index=True)

In [ ]:
code_bing.columns

In [ ]:
def trim_float_num(x):
    #print type(x)
    #print 'x=',x
    xx = 0.000
    if x is None:
        xx = 0.000
    else:
        xx = float(str(x))
    # print xx
    if xx < 0.001:
        return "0.1 %"
    else:
        ret = '%.1f' %(xx * 100.00)
        return ret + ' %'

In [ ]:
trim_profit_rate_x = code_bing['profit_rate_x'].map(trim_float_num)
code_bing['profit_rate_x'] = trim_profit_rate_x
trim_profit_rate_y = code_bing['profit_rate_y'].map(trim_float_num)
code_bing['profit_rate_y'] = trim_profit_rate_y

In [ ]:
code_bing

In [ ]:
code1,code_2,code_all = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))

In [ ]:
code_all[code_all['code']==858]

In [ ]:
code2,code21,code2_all = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))

In [ ]:
code2_all[code2_all['code']==858]

In [ ]:
code21.ix[858]